## Homework

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-08 15:58:51--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 108.177.14.194, 2a00:1450:4010:c0f::c2
Connecting to docs.google.com (docs.google.com)|108.177.14.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/v4c3mkouamn14audi6fca7o2l73i9d2o/1652014725000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-08 15:58:52--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/v4c3mkouamn14audi6fca7o2l73i9d2o/1652014725000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 173.194.222.132, 2a00:1450:4010:c0b::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleuserconten

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [3]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 292 entries, id to price
dtypes: datetime64[ns](1), float64(119), int64(157), object(15)
memory usage: 44.6+ MB
None


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [5]:
print(df.columns)

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price'],
      dtype='object', length=292)


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

Выкинем пока колонки id и timestamp

In [6]:
df_dropped = df.drop(columns=["id", "timestamp"])

In [7]:
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology']
num_columns = list(set(df_dropped.columns).difference(set(cat_columns)))

Заполним пропуски в чиcловых признаках средним, а в категориальных - самым частым.

In [8]:
df_filled = pd.DataFrame(df_dropped, copy=True)
df_filled[num_columns] = SimpleImputer(strategy="mean").fit_transform(df_dropped[num_columns])
df_filled[cat_columns] = SimpleImputer(strategy="most_frequent").fit_transform(df_dropped[cat_columns])

In [9]:
df_filled

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.00000,1.000000,1.0,OwnerOccupier,...,36.0,7.0,2.0,15.0,33.0,1.0,12.0,75.0,10.0,15318960.0
1,64.0,64.000000,16.0,12.603928,1.0,3687.417575,1.91071,6.454917,2.0,OwnerOccupier,...,2.0,2.0,0.0,0.0,13.0,1.0,0.0,6.0,1.0,6080000.0
2,83.0,44.000000,9.0,17.000000,1.0,1985.000000,3.00000,10.000000,3.0,Investment,...,13.0,6.0,1.0,8.0,18.0,0.0,1.0,52.0,0.0,17000000.0
3,71.0,49.000000,2.0,12.603928,1.0,3687.417575,1.91071,6.454917,2.0,Investment,...,0.0,0.0,0.0,1.0,3.0,0.0,2.0,8.0,2.0,990000.0
4,60.0,42.000000,9.0,9.000000,1.0,1970.000000,3.00000,6.000000,2.0,Investment,...,3.0,1.0,0.0,5.0,8.0,0.0,1.0,34.0,5.0,7900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45.0,27.000000,2.0,9.000000,1.0,1978.000000,2.00000,5.000000,2.0,Investment,...,9.0,0.0,0.0,6.0,29.0,0.0,1.0,46.0,10.0,7200000.0
19996,38.0,34.032333,4.0,17.000000,1.0,3687.417575,2.00000,1.000000,1.0,OwnerOccupier,...,2.0,2.0,0.0,1.0,12.0,1.0,0.0,8.0,0.0,3971520.0
19997,35.0,14.000000,9.0,22.000000,1.0,2001.000000,1.00000,10.000000,2.0,Investment,...,13.0,3.0,0.0,6.0,27.0,1.0,10.0,57.0,13.0,7200000.0
19998,51.0,30.000000,8.0,17.000000,1.0,2011.000000,2.00000,9.000000,2.0,Investment,...,1.0,0.0,0.0,0.0,7.0,0.0,0.0,3.0,0.0,3300000.0


Для категориальных признаков применим one-hot кодирование

In [10]:
df_enc = pd.get_dummies(df_filled)

In [11]:
df_enc

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1.0,1.000000,1.0,1.000000,1.000000,1.00000,1.000000,5.293465e+06,77878.0,0.023464,...,0,0,0,0,0,0,0,0,1,0
1,64.0,64.000000,16.0,12.603928,3687.417575,1.91071,6.454917,6.677245e+07,9553.0,0.336177,...,0,0,0,0,0,0,0,1,0,0
2,83.0,44.000000,9.0,17.000000,1985.000000,3.00000,10.000000,1.216448e+07,78507.0,0.297166,...,0,0,0,0,0,0,1,0,0,0
3,71.0,49.000000,2.0,12.603928,3687.417575,1.91071,6.454917,4.708040e+06,38075.0,0.271702,...,0,0,0,0,0,0,0,1,0,0
4,60.0,42.000000,9.0,9.000000,1970.000000,3.00000,6.000000,1.428699e+07,157010.0,0.389354,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45.0,27.000000,2.0,9.000000,1978.000000,2.00000,5.000000,2.481385e+07,174831.0,0.683844,...,0,0,0,0,0,0,1,0,0,0
19996,38.0,34.032333,4.0,17.000000,3687.417575,2.00000,1.000000,6.677245e+07,9553.0,0.336177,...,0,0,0,0,0,0,0,1,0,0
19997,35.0,14.000000,9.0,22.000000,2001.000000,1.00000,10.000000,1.004686e+07,76156.0,0.071056,...,0,0,0,0,0,1,0,0,0,0
19998,51.0,30.000000,8.0,17.000000,2011.000000,2.00000,9.000000,4.036700e+07,21819.0,0.492840,...,0,0,0,0,0,0,0,1,0,0


In [12]:
X = df_enc.drop(columns=["price"])
y = df_enc["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [25]:
from sklearn.tree import DecisionTreeRegressor

In [54]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)

In [56]:
pred

array([11684305.,  5813760., 18000000., ...,  2357000.,  5550000.,
        6800000.])

Проверьте качество на отложенной выборке.

In [61]:
print(mean_squared_error(pred, y_test)**0.5)

3833207.632678462


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [13]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_enc["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_enc["week_year_cnt"] = week_year.map(week_year_cnt_map)

/tmp/ipykernel_174511/3601704785.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [14]:
df_enc["month"] = df.timestamp.dt.month
df_enc["week_day"] = df.timestamp.dt.weekday
df_enc["floor_to_max"] = df_enc["floor"] / df_enc["max_floor"]
df_enc["kitchen_to_full"] = df_enc["kitch_sq"] / df_enc["full_sq"]
df_enc["mean_room"] = df_enc["full_sq"] / df_enc["num_room"]
df_enc["life_to_full"] = df_enc["life_sq"] / df_enc["full_sq"]

Помимо оюязательных были также добавлены: средняяя площадь комнаты и доля жилой площади от общей.

In [15]:
df_enc

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ecology_poor,ecology_satisfactory,month_year_cnt,week_year_cnt,month,week_day,floor_to_max,kitchen_to_full,mean_room,life_to_full
0,1.0,1.000000,1.0,1.000000,1.000000,1.00000,1.000000,5.293465e+06,77878.0,0.023464,...,1,0,1122,112,12,4,1.000000,1.000000,1.000000,1.000000
1,64.0,64.000000,16.0,12.603928,3687.417575,1.91071,6.454917,6.677245e+07,9553.0,0.336177,...,0,0,327,68,10,3,1.269446,0.100858,33.495396,1.000000
2,83.0,44.000000,9.0,17.000000,1985.000000,3.00000,10.000000,1.216448e+07,78507.0,0.297166,...,0,0,752,192,2,2,0.529412,0.120482,27.666667,0.530120
3,71.0,49.000000,2.0,12.603928,3687.417575,1.91071,6.454917,4.708040e+06,38075.0,0.271702,...,0,0,262,69,7,3,0.158681,0.090914,37.158955,0.690141
4,60.0,42.000000,9.0,9.000000,1970.000000,3.00000,6.000000,1.428699e+07,157010.0,0.389354,...,0,0,711,214,10,2,1.000000,0.100000,20.000000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45.0,27.000000,2.0,9.000000,1978.000000,2.00000,5.000000,2.481385e+07,174831.0,0.683844,...,0,0,734,181,9,0,0.222222,0.111111,22.500000,0.600000
19996,38.0,34.032333,4.0,17.000000,3687.417575,2.00000,1.000000,6.677245e+07,9553.0,0.336177,...,0,0,579,153,12,4,0.235294,0.026316,19.000000,0.895588
19997,35.0,14.000000,9.0,22.000000,2001.000000,1.00000,10.000000,1.004686e+07,76156.0,0.071056,...,0,0,861,277,6,3,0.409091,0.285714,35.000000,0.400000
19998,51.0,30.000000,8.0,17.000000,2011.000000,2.00000,9.000000,4.036700e+07,21819.0,0.492840,...,0,0,573,164,1,1,0.470588,0.176471,25.500000,0.588235


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [16]:
df_enc.isnull().any(axis=1).mean()

0.00015

In [17]:
np.array(df_enc == np.inf).any(axis=1).shape

(20000,)

In [18]:
df_enc = df_enc.dropna(axis=0)

In [ ]:
m = df_enc.loc[df_enc["floor_to_max"] != np.inf, "floor_to_max"].max()
df_enc["floor_to_max"].replace(np.inf, m, inplace=True)
m = df_enc.loc[df_enc["kitchen_to_full"] != np.inf, "kitchen_to_full"].max()
df_enc["kitchen_to_full"].replace(np.inf, m, inplace=True)
m = df_enc.loc[df_enc["mean_room"] != np.inf, "mean_room"].max()
df_enc["mean_room"].replace(np.inf, m, inplace=True)
m = df_enc.loc[df_enc["life_to_full"] != np.inf, "life_to_full"].max()
df_enc["life_to_full"].replace(np.inf, m, inplace=True)

In [20]:
X = df_enc.drop(columns=["price"])
y = df_enc["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

### Дерево решений

In [93]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
print(mean_squared_error(pred, y_test)**0.5)

3630828.0567652574


Как видно, путем feature engineering удалось немного повысить качество предсказаний

In [100]:
tree = DecisionTreeRegressor()
parameters = {"max_depth":[2, 10, 50, 100, 200], "min_samples_leaf":[1, 5, 10, 100, 1000]}
gs_tree = GridSearchCV(tree,
                      param_grid=parameters,
                      scoring="neg_mean_squared_error",
                      cv=5)
gs_tree.fit(X_train, y_train)
gs_tree.best_params_

{'max_depth': 10, 'min_samples_leaf': 100}

In [104]:
tree = DecisionTreeRegressor(max_depth=10, min_samples_leaf=100)
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
print(mean_squared_error(pred, y_test)**0.5)

3039220.0369040035


За счет подбора параметров удалось значительно улучшить какчество

### Случайный лес

In [109]:
import multiprocessing
from sklearn.ensemble import RandomForestRegressor
workers = multiprocessing.cpu_count() - 1

In [118]:
forest = RandomForestRegressor(n_jobs=workers)
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)
print(mean_squared_error(forest_pred, y_test)**0.5)

2704279.3837902145


Как видно лес с параметрами по-умолчанию справляется сильно лучше одного дерева, попробуем перебрать некоторые параметры

In [116]:
forest = RandomForestRegressor(min_samples_leaf=100, random_state=42, n_jobs=workers)
parameters = {"n_estimators": [100, 200], "max_depth": [5, 10, 20], "max_features": [0.2, 0.333, "sqrt", "auto"], "max_samples": [0.1, 0.2, 0.5, None]}
gs_forest = GridSearchCV(forest,
                      param_grid=parameters,
                      scoring="neg_mean_squared_error",
                      cv=5)
gs_forest.fit(X_train, y_train)
gs_forest.best_params_

{'max_depth': 20,
 'max_features': 'auto',
 'max_samples': None,
 'n_estimators': 200}

In [117]:
forest = RandomForestRegressor(max_depth=20, max_features="auto", max_samples=None, n_estimators=200, min_samples_leaf=100, n_jobs=workers)
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)
print(mean_squared_error(forest_pred, y_test)**0.5)

2978504.8092333404


Как видно, среди параметров не оказалось, лучше тех, что по-умолчанию. Требуется более скрупулезный их перебор. В связи с довольно длительным обучением (и надвигающейся сессией) перебор был остановлен на этом.

### Бустинг

In [21]:
from catboost import CatBoostRegressor
boost = CatBoostRegressor(loss_function="RMSE", verbose=False)
boost.fit(X_train, y_train)
boost_pred = boost.predict(X_test)
print(mean_squared_error(boost_pred, y_test)**0.5)

2593981.066253547


Как видно catboost с параметрвами по-умолчанию справляется лучше предыдущих моделей.

In [ ]:
boost = CatBoostRegressor(loss_function="RMSE", random_state=42, silent=True)
parameters = {"iterations": [10, 20, 100, 200], "learning_rate": [0.02, 0.05, 0.1], "depth": [3, 5, 7]}
result = boost.grid_search(parameters, X=X_train, y=y_train, plot=True, verbose=False)

In [28]:
result["params"]

{'depth': 7, 'iterations': 200, 'learning_rate': 0.1}

In [29]:
from catboost import CatBoostRegressor
boost = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7, iterations=200, learning_rate=0.1, verbose=False)
boost.fit(X_train, y_train)
boost_pred = boost.predict(X_test)
print(mean_squared_error(boost_pred, y_test)**0.5)

2651675.1410435736


Ситуация аналогична лесу, требуется более скрупулезный перебор, на который нужно время...

In [30]:
from catboost import CatBoostRegressor
boost = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7,
                          iterations=500, learning_rate=0.15, verbose=False)
boost.fit(X_train, y_train)
boost_pred = boost.predict(X_test)
print(mean_squared_error(boost_pred, y_test)**0.5)

2571387.9225949896


Методом научного тыка удалось немного улучшить параметры, результат превзошел модель "из коробки".

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [ ]:
X_train.loc[:, "price"] = y_train
invest = X_train.loc[X_train["product_type_Investment"] == 1, :]
invest_X_train = invest.drop(columns="price")
invest_y_train = invest["price"]
owner = X_train.loc[X_train["product_type_OwnerOccupier"] == 1, :]
owner_X_train = owner.drop(columns="price")
owner_y_train = owner["price"]

In [164]:
boost_invest = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7,
                          iterations=500, learning_rate=0.15, verbose=False)
boost_invest.fit(invest_X_train, invest_y_train)
boost_invest_pred = boost_invest.predict(X_test[X_test["product_type_Investment"] == 1])

boost_owner = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7,
                          iterations=500, learning_rate=0.15, verbose=False)
boost_owner.fit(owner_X_train, owner_y_train)
boost_owner_pred = boost_owner.predict(X_test[X_test["product_type_OwnerOccupier"] == 1])

In [181]:
boost_invest_pred_indexed = pd.Series(boost_invest_pred, index=X_test[X_test["product_type_Investment"] == 1].index)
boost_owner_pred_indexed = pd.Series(boost_owner_pred, index=X_test[X_test["product_type_OwnerOccupier"] == 1].index)
prediction = pd.concat((boost_invest_pred_indexed, boost_owner_pred_indexed))
prediction = prediction[y_test.index]
print(mean_squared_error(prediction, y_test)**0.5)

2543698.88206796


Как видно, модели обученные на стратифицированных данных (следовательно, на выборкахх с меньшей дисперсией) показали себя лучше, чем одна модель

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [211]:
from sklearn.linear_model import HuberRegressor
huber = HuberRegressor(max_iter=3000).fit(X=invest_X_train, y=invest_y_train)
huber_pred = huber.predict(X_test[X_test["product_type_Investment"] == 1])

In [212]:
rmses = []
for huber_w in np.arange(0, 1.1, 0.1):
    boost_w = 1 - huber_w
    total_pred = huber_pred * huber_w + boost_invest_pred * boost_w
    y_test_invest = y_test[X_test["product_type_Investment"] == 1]
    y_test_invest = y_test_invest[X_test[X_test["product_type_Investment"] == 1].index]
    rmse = mean_squared_error(total_pred, y_test_invest)**0.5
    print(rmse)
    rmses.append(rmse)

3002620.874375416
8355234.900689829
15927970.612124339
23678822.28575164
31476484.576652482
39293098.84970635
47119234.2103081
54950822.80879392
62785824.049745336
70623102.13586074
78461974.79303826


Huber regression - линейная модель с повышенной устойчивочтью к выбросам. Как видно, она катастрофически плохо описывает данные. Вероятнее всего необходим другой подход к подготовке данных, feature engineering.